# Supercomp - Atividade 12

## Funções Customizadas e Otimização com Fusion Kernel

In [1]:
!nvidia-smi

Fri Oct 11 12:31:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Introdução a Operações Customizadas

Implemente o cálculo Saxpy para dois vetores aleatórios, experimente diferentes valores para a constante a e observe os resultados.

In [2]:
%%writefile operacao_customizada.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <cstdlib> // Para rand()
#include <iostream>
using namespace std;

struct saxpy
{
    int a;
    saxpy(int a_) : a(a_) {}  // Inicializa a constante a

    __host__ __device__
    double operator()(const int& x, const int& y) const {
        return a * x + y;  // Aplica a fórmula a * x + y
    }
};

int main() {
    const int N = 1000;

    thrust::host_vector<int> h_a(N);
    thrust::host_vector<int> h_b(N);

    for (int i = 0; i < N; ++i) {
        h_a[i] = rand() % 100;
        h_b[i] = rand() % 100;
    }

    thrust::device_vector<int> d_a = h_a;
    thrust::device_vector<int> d_b = h_b;
    thrust::device_vector<double> d_c(N);

    int a = 5;

    thrust::transform(d_a.begin(), d_a.end(), d_b.begin(), d_c.begin(), saxpy(a));

    thrust::host_vector<double> h_c = d_c;

    for (int i = 0; i < 10; ++i) { cout << "Resultado [" << i << "]: " << h_c[i] << endl;
    }

    return 0;
}

Writing operacao_customizada.cu


In [3]:
!nvcc -arch=sm_75 -std=c++14 operacao_customizada.cu -o operacao_customizada

In [4]:
!./operacao_customizada

Resultado [0]: 501
Resultado [1]: 400
Resultado [2]: 500
Resultado [3]: 522
Resultado [4]: 266
Resultado [5]: 337
Resultado [6]: 509
Resultado [7]: 341
Resultado [8]: 226
Resultado [9]: 396


## Transformações Unárias e Binárias

In [5]:
%%writefile unaria_binaria.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <cstdlib>
#include <iostream>
using namespace std;


struct square
{
    __host__ __device__
    float operator()(const float& x) const {
        return x * x;
    }
};


int main(){
    const int N = 1000;
    thrust::host_vector<float> h_v(N);
    for (int i = 0; i < N; ++i) {
        h_v[i] = rand() % 10;
    }

    thrust::device_vector<float> d_v = h_v;
    thrust::transform(d_v.begin(), d_v.end(), d_v.begin(), square());
    thrust::host_vector<float> h_result = d_v;

    for (int i = 0; i < 10; ++i) {
        cout << "Resultado [" << i << "]: " << h_result[i] << endl;
    }
    return 0;
}


Writing unaria_binaria.cu


In [6]:
!nvcc -arch=sm_75 -std=c++14 unaria_binaria.cu -o unaria_binaria

In [7]:
!./unaria_binaria

Resultado [0]: 9
Resultado [1]: 36
Resultado [2]: 49
Resultado [3]: 25
Resultado [4]: 9
Resultado [5]: 25
Resultado [6]: 36
Resultado [7]: 4
Resultado [8]: 81
Resultado [9]: 1


## Calculando a Magnitude de um Vetor

Implemente uma função que calcula a magnitude de um vetor de floats. Compare os resultados com uma implementação em C++ feita na CPU para ver a diferença no tempo de execução.

In [16]:
%%writefile magnitude.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/transform_reduce.h>
#include <cstdlib>
#include <iostream>
#include <cmath>
#include <chrono>
using namespace std;

struct square
{
    __host__ __device__
    double operator()(const double& x) const {
        return x * x;
    }
};

double magnitude_gpu(thrust::device_vector<double>& v) {
    double sum_of_squares = thrust::transform_reduce(v.begin(), v.end(), square(), 0.0f, thrust::plus<double>());
    return std::sqrt(sum_of_squares);
}

double magnitude_cpu(const std::vector<double>& v) {
    double sum_of_squares = 0.0f;
    for (size_t i = 0; i < v.size(); ++i) {
        sum_of_squares += v[i] * v[i];
    }
    return std::sqrt(sum_of_squares);
}

int main() {
    const int N = 1000000;  // Tamanho do vetor
    thrust::host_vector<double> h_v(N);

    for (int i = 0; i < N; ++i) { h_v[i] = static_cast<double>(rand() % 10); }

    thrust::device_vector<double> d_v = h_v;

    // Conversão para std::vector para a implementação na CPU
    std::vector<double> v_cpu(h_v.begin(), h_v.end());

    auto start_gpu = chrono::high_resolution_clock::now();
    double result_gpu = magnitude_gpu(d_v);
    auto end_gpu = chrono::high_resolution_clock::now();
    chrono::duration<double> duration_gpu = end_gpu - start_gpu;

    auto start_cpu = chrono::high_resolution_clock::now();
    double result_cpu = magnitude_cpu(v_cpu);
    auto end_cpu = chrono::high_resolution_clock::now();
    chrono::duration<double> duration_cpu = end_cpu - start_cpu;

    cout << "Magnitude GPU: " << result_gpu << endl;
    cout << "Tempo GPU: " << duration_gpu.count() << " segundos" << endl;

    cout << "Magnitude CPU: " << result_cpu << endl;
    cout << "Tempo CPU: " << duration_cpu.count() << " segundos" << endl;

    return 0;
}

Overwriting magnitude.cu


In [17]:
!nvcc -arch=sm_75 -std=c++14 magnitude.cu -o magnitude

In [18]:
!./magnitude

Magnitude GPU: 5340.79
Tempo GPU: 0.000865067 segundos
Magnitude CPU: 5340.79
Tempo CPU: 0.00732827 segundos


## Otimização com Fusion Kernel

Implemente o cálculo da variância usando a técnica de fusion kernel. Compare o desempenho com a implementação que calcula a média e a variância em etapas separadas. Use diferentes tamanhos de vetor e observe as diferenças de desempenho.

In [11]:
%%writefile variancia_fusion_kernel.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/transform_reduce.h>
#include <cstdlib>
#include <iostream>
#include <cmath>
#include <chrono>
using namespace std;

struct variance_op
{
    float mean;
    variance_op(float mean_) : mean(mean_) {}

    __host__ __device__
    float operator()(const float& x) const {
        float diff = x - mean;
        return diff * diff;
    }
};

struct variance_fusion_op
{
    __host__ __device__
    thrust::pair<float, float> operator()(const float& x) const {
        return thrust::make_pair(x, x * x);  // Retorna (x, x^2)
    }
};

float calculate_mean(const thrust::device_vector<float>& d_vec) { return thrust::reduce(d_vec.begin(), d_vec.end(), 0.0f, thrust::plus<float>()) / d_vec.size(); }

float calculate_variance(const thrust::device_vector<float>& d_vec, float mean) {
    return thrust::transform_reduce(d_vec.begin(), d_vec.end(), variance_op(mean), 0.0f, thrust::plus<float>()) / d_vec.size();
}

float calculate_variance_fusion(const thrust::device_vector<float>& d_vec) {
    thrust::pair<float, float> result = thrust::transform_reduce(d_vec.begin(), d_vec.end(), variance_fusion_op(), thrust::make_pair(0.0f, 0.0f),
          [] __host__ __device__ (const thrust::pair<float, float>& a, const thrust::pair<float, float>& b) {
        return thrust::make_pair(a.first + b.first, a.second + b.second);  // Soma de (x, x^2)
    });

    float mean = result.first / d_vec.size();
    float mean_of_squares = result.second / d_vec.size();
    return mean_of_squares - mean * mean;
}

int main(){
    const int N = 1000000;
    thrust::host_vector<float> h_v(N);

    for (int i = 0; i < N; ++i) { h_v[i] = rand() % 10; }

    thrust::device_vector<float> d_v = h_v;

    auto start = chrono::high_resolution_clock::now();
    float mean = calculate_mean(d_v);
    float variance_traditional = calculate_variance(d_v, mean);
    auto end = chrono::high_resolution_clock::now();
    chrono::duration<float> duration_traditional = end - start;

    start = chrono::high_resolution_clock::now();
    float variance_fusion = calculate_variance_fusion(d_v);
    end = chrono::high_resolution_clock::now();
    chrono::duration<float> duration_fusion = end - start;

    cout << "Variância tradicional: " << variance_traditional << endl;
    cout << "Tempo variância tradicional: " << duration_traditional.count() << " segundos" << endl;

    cout << "Variância com Fusion Kernel: " << variance_fusion << endl;
    cout << "Tempo variância Fusion Kernel: " << duration_fusion.count() << " segundos" << endl;

    return 0;
}

Writing variancia_fusion_kernel.cu


In [14]:
!nvcc -arch=sm_75 -std=c++14 --extended-lambda variancia_fusion_kernel.cu -o variancia_fusion_kernel

In [15]:
!./variancia_fusion_kernel

Variância tradicional: 8.24941
Tempo variância tradicional: 0.00118417 segundos
Variância com Fusion Kernel: 8.24941
Tempo variância Fusion Kernel: 0.000252958 segundos
